In [1]:
import pandas as pd
import numpy as np
import os
os.chdir(r'../data')
from tqdm import tqdm
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore')
def function_that_warns():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    function_that_warns()  # this will not show a warning

In [2]:
import spacy
nlp_trf = spacy.load('en_core_web_trf')

In [3]:
ClinNotes_df = pd.read_pickle('ClinNotes_entity_df')
print(ClinNotes_df.columns, ClinNotes_df.shape)

Index(['category', 'notes', 'disease', 'chemical'], dtype='object') (100, 4)


In [4]:
# def get_disease_chemical(clinical_note_text):
#     sentence = Sentence(clinical_note_text, use_tokenizer=SciSpacyTokenizer())
#     tagger.predict(sentence)
#     disease = [disease.text for disease in sentence.get_spans("hunflair-disease")]
#     chemical = [chemical.text for chemical in sentence.get_spans("hunflair-chemical")]
#     return disease, chemical
def get_entities_from_clinical_note(clinical_note):
    document = nlp_trf(clinical_note)
    DATE_TIME = [ent.text for ent in document.ents if ent.label_ in  ['DATE','TIME']]
    QUANTITY = [ent.text for ent in document.ents if ent.label_ in  ['QUANTITY']]
    ORG = [ent.text for ent in document.ents if ent.label_ in  ['ORG']] 
    Person = [ent.text for ent in document.ents if ent.label_ in  ['PERSON']] 
    Location = [ent.text for ent in document.ents if ent.label_ in  ['GPE']]
    return list(set(DATE_TIME)), list(set(QUANTITY)), list(set(ORG)), list(set(Person)), list(set(Location))

In [5]:
ClinNotes_df['entity_tagging'] = ClinNotes_df['notes'].progress_apply(get_entities_from_clinical_note)
ClinNotes_df.head()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [08:35<00:00,  5.15s/it]


,category,notes,disease,chemical,entity_tagging
0,Cardiovascular / Pulmonary,"REASON FOR CONSULTATION: , Pulmonary embolism....","[syncopal episode, shortness of breath, chest ...","[Aspirin.,2 . Potassium.,3 . Lasix.,4 . ...","([night, one year ago, yesterday, the last sev..."
1,Cardiovascular / Pulmonary,"PREOPERATIVE DIAGNOSIS:, Airway stenosis with...","[Airway stenosis, Airway stenosis, bleeding, t...",[silicon],"([two hours], [3 cm, 2.25 cm, 9 cm, 14 mm, 18 ..."
2,Cardiovascular / Pulmonary,"PROCEDURE DONE: ,Resting Myoview and adenosin...","[dizziness, shortness of breath, arrhythmias]","[adenosine, Tc-99 m tetrofosmin, adenosine, ad...","([Two minutes, four minutes], [90 beats, 31.6 ..."
3,Gastroenterology,"PREPROCEDURE DIAGNOSIS:, Change in bowel func...","[hematochezia, ulcerations, edema, mucosal abn...",[],"([2 weeks, a 33-year-old], [], [], [Sims], [])"
4,Neurology,"HISTORY OF PRESENT ILLNESS: ,The patient is a...","[right arm pain, pain, dull pain, achy sensati...","[Medrol, ACTOplus, albuterol, AndroGel, Asteli...","([2006, 1984, 89, a little less than a year ag..."


In [6]:
ClinNotes_df['DATE_TIME'] = ClinNotes_df['entity_tagging'].apply(lambda x: x[0])
ClinNotes_df['QUANTITY'] = ClinNotes_df['entity_tagging'].apply(lambda x: x[1])
ClinNotes_df['ORG'] = ClinNotes_df['entity_tagging'].apply(lambda x: x[2])
ClinNotes_df['Person'] = ClinNotes_df['entity_tagging'].apply(lambda x: x[3])
ClinNotes_df['Location'] = ClinNotes_df['entity_tagging'].apply(lambda x: x[4])
del ClinNotes_df['entity_tagging']

In [7]:
ClinNotes_df['DATE_TIME'] = ClinNotes_df['DATE_TIME'].apply(lambda y: np.nan if len(y)==0 else y)
ClinNotes_df['QUANTITY'] = ClinNotes_df['QUANTITY'].apply(lambda y: np.nan if len(y)==0 else y)
ClinNotes_df['ORG'] = ClinNotes_df['ORG'].apply(lambda y: np.nan if len(y)==0 else y)
ClinNotes_df['Person'] = ClinNotes_df['Person'].apply(lambda y: np.nan if len(y)==0 else y)
ClinNotes_df['Location'] = ClinNotes_df['Location'].apply(lambda y: np.nan if len(y)==0 else y)
ClinNotes_df['disease'] = ClinNotes_df['disease'].apply(lambda y: np.nan if len(y)==0 else y)
ClinNotes_df['chemical'] = ClinNotes_df['chemical'].apply(lambda y: np.nan if len(y)==0 else y)

In [8]:
print(ClinNotes_df.shape, ClinNotes_df.columns)
ClinNotes_df.head()

(100, 9) Index(['category', 'notes', 'disease', 'chemical', 'DATE_TIME', 'QUANTITY',
       'ORG', 'Person', 'Location'],
      dtype='object')


,category,notes,disease,chemical,DATE_TIME,QUANTITY,ORG,Person,Location
0,Cardiovascular / Pulmonary,"REASON FOR CONSULTATION: , Pulmonary embolism....","[syncopal episode, shortness of breath, chest ...","[Aspirin.,2 . Potassium.,3 . Lasix.,4 . ...","[night, one year ago, yesterday, the last seve...",[2 to 3 glasses],NaN,NaN,NaN
1,Cardiovascular / Pulmonary,"PREOPERATIVE DIAGNOSIS:, Airway stenosis with...","[Airway stenosis, Airway stenosis, bleeding, t...",[silicon],[two hours],"[3 cm, 2.25 cm, 9 cm, 14 mm, 18 mm]",NaN,"[D, Donovan]",NaN
2,Cardiovascular / Pulmonary,"PROCEDURE DONE: ,Resting Myoview and adenosin...","[dizziness, shortness of breath, arrhythmias]","[adenosine, Tc-99 m tetrofosmin, adenosine, ad...","[Two minutes, four minutes]","[90 beats, 31.6 mCi, 38 mg, 13.3 mCi, 140 mcg,...",NaN,NaN,NaN
3,Gastroenterology,"PREPROCEDURE DIAGNOSIS:, Change in bowel func...","[hematochezia, ulcerations, edema, mucosal abn...",NaN,"[2 weeks, a 33-year-old]",NaN,NaN,[Sims],NaN
4,Neurology,"HISTORY OF PRESENT ILLNESS: ,The patient is a...","[right arm pain, pain, dull pain, achy sensati...","[Medrol, ACTOplus, albuterol, AndroGel, Asteli...","[2006, 1984, 89, a little less than a year ago...",[81 mg],"[the State of Maryland, Human Resources, Spine...","[X, Hoffman]",NaN


In [10]:
ClinNotes_df.to_pickle('ClinNotes_entity_phase2_df')